In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import urllib.parse
import gzip
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chi2_contingency
import env
import os
import pandas as pd
from sqlalchemy import create_engine
from io import BytesIO
from io import StringIO
from env import user, hostname, password 

In [2]:
def get_connection(db, user=env.user, hostname=env.hostname, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{hostname}/{db}'

In [5]:
def get_sql_data():
    '''
    If the CSV file exists, it is read and returned as a pandas DataFrame.
    If not, pandas reads in a SQL query that acquires log data from a MySQL database.
    The query is stored into a DataFrame, saved, and returned.
    '''
    filename = 'curriculum_logs.csv'
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # Define SQL query, database parameters, and filename
        sql_query = """
            SELECT l.date, l.time, l.path as lesson, l.user_id, c.name, c.program_id,
                   l.ip, c.start_date, c.end_date
            FROM logs l
            JOIN cohorts c ON c.id=l.cohort_id;
        """
        db = 'curriculum_logs'
        user = env.user  # Replace with the actual value
        hostname = env.hostname  # Replace with the actual value
        password = env.password  # Replace with the actual value
        
        # Create a database connection using SQLAlchemy
        connection_string = get_connection(db, user, hostname, password)
        engine = create_engine(connection_string)
        
        # Fetch data from the database and convert it into a DataFrame
        logs_df = pd.read_sql(sql_query, get_connection('curriculum_logs'))
        
        # Convert 'date' and 'time' columns to a single 'datetime' column
        logs_df['date'] = pd.to_datetime(logs_df['date'] + ' ' + logs_df['time'])
        logs_df = logs_df.drop(['time'], axis=1)  # Drop 'time' column
        logs_df = logs_df.set_index('date')
        
        # Save the DataFrame as a CSV file
        logs_df.to_csv(filename, index=False)
        
        return logs_df

In [6]:
# Call the function to get and save the data
logs_df = get_sql_data()

In [7]:
logs_df.head()

,date,time,lesson,user_id,name,program_id,ip,start_date,end_date
0,2018-01-26,09:55:03,/,1,Hampton,1,97.105.19.61,2015-09-22,2016-02-06
1,2018-01-26,09:56:02,java-ii,1,Hampton,1,97.105.19.61,2015-09-22,2016-02-06
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,Hampton,1,97.105.19.61,2015-09-22,2016-02-06
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,Hampton,1,97.105.19.61,2015-09-22,2016-02-06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,Teddy,2,97.105.19.61,2018-01-08,2018-05-17
